# Case Datarisk - Detecção de Fraudes

Foram fornecidas os arquivos base_info.csv, base_cadastral.csv e base_pagamentos.csv.
- base_cadastral possui informações fixas (cadastrais) sobre cada cliente.
- base_info possui informações mensais de renda e número de empregados.
- base_pagamentos possui informações referentes a cada operação de crédito.

## Preprocessamento e Visualização dos Dados
- leitura dos arquivos e ajuste de cada tipo de dado.
- codificação (one_hot, categorica e ordinal) dos dados cadastrais.
- visualização dos dados do cliente reduzido para 3 dimensões.

In [ ]:
## Leitura dos arquivos e ajuste de cada tipo de dado.

import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot  as plt
import seaborn as sns

# Load the data from the CSV files
base_info = pd.read_csv('base_info.csv')
base_pagamentos = pd.read_csv('base_pagamentos_desenvolvimento.csv')
base_pagamentos_teste = pd.read_csv('base_pagamentos_teste.csv')
base_cadastral = pd.read_csv('base_cadastral.csv')

# lower case all column names
base_info.columns = base_info.columns.str.lower()
base_pagamentos.columns = base_pagamentos.columns.str.lower()
base_pagamentos_teste.columns = base_pagamentos_teste.columns.str.lower()
base_cadastral.columns = base_cadastral.columns.str.lower()

# give a new id to each client based on registrartion date order
base_cadastral = base_cadastral.sort_values('data_cadastro').reset_index(drop=True)
base_cadastral['new_id'] = range(1, len(base_cadastral) + 1)

# save a corresponding dictionary of old_id and new_id
id_dict = base_cadastral[['id_cliente', 'new_id']].set_index('id_cliente').to_dict()

# convert str to date
def convert_date(df, to_date_columns, format):
  df_copy = df.copy()
  for col in to_date_columns:
    df_copy[col] = pd.to_datetime(df_copy[col], format=format)#.dt.date
  return df_copy

date_columns = ['data_emissao_documento', 'data_pagamento', 'data_vencimento']
base_pagamentos_date = convert_date(base_pagamentos, date_columns, '%Y-%m-%d')
base_pagamentos_date = convert_date(base_pagamentos_date, ['safra_ref'], '%Y-%m')
date_columns = ['data_emissao_documento', 'data_vencimento']
base_pagamentos_teste_date = convert_date(base_pagamentos_teste, date_columns, '%Y-%m-%d')
base_pagamentos_teste_date = convert_date(base_pagamentos_teste_date, ['safra_ref'], '%Y-%m')
base_info_date = convert_date(base_info, ['safra_ref'], '%Y-%m')
base_cadastral_date = convert_date(base_cadastral, ['data_cadastro'], '%Y-%m-%d')

# ajust values in base_cadastral
base_cadastral_date.flag_pf = (base_cadastral_date.flag_pf == 'X').astype(int)
base_cadastral_date.segmento_industrial = base_cadastral_date.segmento_industrial.fillna('NAN')
base_cadastral_date.dominio_email = base_cadastral_date.dominio_email.fillna('NAN')
base_cadastral_date.porte = base_cadastral_date.porte.fillna('NAN')
base_cadastral_date.cep_2_dig = base_cadastral_date.cep_2_dig.fillna('NA')
base_cadastral_date.ddd = base_cadastral_date.ddd.fillna('NAN')
base_cadastral_date.loc[base_cadastral_date['ddd'].str.contains("\("), 'ddd'] = 'INVALID'

# Add a binary column, fraud, to base_pagamentos_desenvolvimento_coherent and assing 1 to rows where DATA_PAGAMENTO > DATA_VENCIMENTO + 5
base_pagamentos_date['late_payment'] = (base_pagamentos_date['data_pagamento'] - base_pagamentos_date['data_vencimento']).dt.days
base_pagamentos_date['fraud'] = np.where(base_pagamentos_date['late_payment'] > 5, 1, 0)

In [ ]:
## codificação (one_hot, categorica e ordinal) dos dados da base cadastral

# date encoding
def encode_date(df, columns):
    df_copy = df.copy()
    for col in columns:
        min_date = df_copy[col].min()
        df_copy[col + '_since_min'] = (df_copy[col] - min_date).dt.days + 1
        df_copy.drop(col, inplace=True, axis=1)
    return df_copy

# One-hot encoding for categorical variables
def encode_categorical(df, columns):
    df_copy = df.copy()
    for col in columns:
        df_copy = pd.concat([df_copy, pd.get_dummies(df_copy[col], prefix=col)], axis=1)
        df_copy.drop(col, inplace=True, axis=1)
    return df_copy

# ordinal encoding for categorical variables
def encode_ordinal(df, columns, mapping):
    df_copy = df.copy()
    for i, col in enumerate(columns):
        df_copy[col + '_encoded'] = df_copy[col].map(mapping[i])
        df_copy.drop(col, inplace=True, axis=1)
    return df_copy

ordinal_encode_map = [{'NAN': 0, 'PEQUENO': 1, 'MEDIO': 2, 'GRANDE': 3}]
base_cadastral_date_encoded = encode_date(base_cadastral_date, ['data_cadastro'])
base_cadastral_one_hot_encoded = encode_categorical(base_cadastral_date_encoded, ['segmento_industrial', 'dominio_email'])
base_cadastral_ordinal_encoded = encode_ordinal(base_cadastral_one_hot_encoded, ['porte'], ordinal_encode_map)
base_cadastral_droped = base_cadastral_ordinal_encoded.drop(['id_cliente', 'ddd', 'cep_2_dig'], axis=1)
base_cadastral_droped

# standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
base_cadastral_scaled = scaler.fit_transform(base_cadastral_droped)
base_cadastral_scaled = pd.DataFrame(base_cadastral_scaled, columns=base_cadastral_droped.columns)
# display(base_cadastral_scaled.head())

In [ ]:
# visualização dos dados do cliente reduzido para 3 dimensões

from sklearn.manifold import TSNE
base_cadastral_tsne = TSNE(n_components=3).fit_transform(base_cadastral_scaled)
print(base_cadastral_tsne.shape)

# plot 16 TSNEs with different columns as color
fig = plt.figure(figsize=(20, 20))
for i, col in enumerate(base_cadastral_scaled.columns):
    ax = fig.add_subplot(4, 4, i+1, projection='3d')
    ax.scatter(base_cadastral_tsne[:, 0], 
               base_cadastral_tsne[:, 1], 
               base_cadastral_tsne[:, 2], 
               c=base_cadastral_scaled[col], 
               cmap='coolwarm')
    plt.title(col)
plt.show()

base_cadastral_tsne_df = pd.DataFrame(base_cadastral_tsne, columns=['tsne_1', 'tsne_2', 'tsne_3'])

## Codificação (compressão) da informação de cada cliente na base cadastral
- codificação via modelo autoencoder para redução de dimensionalidade de 15 para 3.
- visualização do treinamento do modelo.
- análise de erro do modelo.
- visualização do cliente em 3 dimensões.

In [ ]:
## codificação via modelo autoencoder para redução de dimensionalidade de 15 para 3

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import BatchNormalization

def create_layers(input_layer, units, activation='relu', dropout_rate=0.1):
    x_dense = Dense(units, activation=activation)(input_layer)
    x_bn = BatchNormalization()(x_dense)
    x_drop = Dropout(dropout_rate)(x_bn)
    return [x_dense, x_bn, x_drop]

def autoencoder_model(input_dim, encoding_dim, learning_rate=0.002, weight_decay=0.0001):
    input_layer = Input(shape=(input_dim, ))

    # encoder
    encoder = create_layers(input_layer, 25)
    encoder = create_layers(encoder[2], 15)
    encoder = create_layers(encoder[2] + input_layer, 15)
    encoder_skip = create_layers(encoder[2], 15)
    encoder = create_layers(encoder_skip[2] + encoder[1], 15)
    encoded = Dense(encoding_dim, activation='linear')(encoder[2])

    # decoder
    decoder = create_layers(encoded, 15)
    decoder_skip = create_layers(decoder[2], 15)
    decoder = create_layers(decoder_skip[2] + decoder[1], 15)
    decoder = create_layers(decoder[2], 15)
    decoder = create_layers(decoder[2], 25)
    decoded = Dense(input_dim, activation='linear')(decoder[2])

    autoencoder = Model(inputs=input_layer, outputs=decoded)

    adam = Adam(learning_rate=learning_rate, weight_decay=weight_decay)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error', metrics=['mae'])

    return autoencoder

input_dim = base_cadastral_scaled.shape[1]
encoding_dim = 3

autoencoder = autoencoder_model(input_dim, encoding_dim)
histories = []
history = autoencoder.fit(base_cadastral_scaled, base_cadastral_scaled, epochs=100, batch_size=64, validation_split=0.2)
histories.append(history)

encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-18].output)
encoded_data = encoder.predict(base_cadastral_scaled)


In [ ]:
## visualização do treinamento do modelo

# plot losses
fig = plt.figure(figsize=(20, 20))
for i, history in enumerate(histories):
    ax = fig.add_subplot(4, 4, i+1)
    ax.plot(history.history['loss'])
    ax.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
plt.show()

encoder.summary()

In [ ]:
## análise de erro do modelo

decoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-1].output)
decoded_data = decoder.predict(base_cadastral_scaled)

decoded_data = pd.DataFrame(decoded_data, columns=base_cadastral_scaled.columns)
decoded_data['error'] = np.reshape(np.mean(np.abs(base_cadastral_scaled.values - decoded_data.values), axis=1), (-1, 1))

decoded_data['error'].hist(bins=100)
plt.show()

col = 'segmento_industrial'
base_cadastral_raw_drop = base_cadastral_date.drop(['ddd', 'cep_2_dig'], axis=1)
display(base_cadastral_raw_drop[decoded_data['error'] > 0.6].groupby('segmento_industrial').count() / base_cadastral_raw_drop.groupby('segmento_industrial').count())
display(base_cadastral_raw_drop[decoded_data['error'] > 0.6].groupby('dominio_email').count() / base_cadastral_raw_drop.groupby('dominio_email').count())
display(base_cadastral_raw_drop[decoded_data['error'] > 0.6].groupby('flag_pf').count() / base_cadastral_raw_drop.groupby('flag_pf').count())
display(base_cadastral_raw_drop[decoded_data['error'] > 0.6].groupby('porte').count() / base_cadastral_raw_drop.groupby('porte').count())

In [ ]:
## visualização do cliente em 3 dimensões

# check uniqueness of each encoded representation
encoded_data_df = pd.DataFrame(encoded_data, columns=['encoded_1', 'encoded_2', 'encoded_3'])
if encoded_data_df.groupby(['encoded_1', 'encoded_2', 'encoded_3']).size().shape[0] / encoded_data_df.shape[0]:
    print('All encoded representations are unique.')
else:
    print('There are repeated encoded representations.')
    print(encoded_data_df
            .groupby(['encoded_1', 'encoded_2', 'encoded_3'])
            .size()
            .reset_index()
            .rename(columns={0:'count'})
            .sort_values(by='count', ascending=False))


# check if the encoded data is separable unsing 16 plots
fig = plt.figure(figsize=(20, 20))
for i, col in enumerate(base_cadastral_scaled.columns):
    ax = fig.add_subplot(4, 4, i+1, projection='3d')
    ax.scatter(encoded_data_df['encoded_1'], encoded_data_df['encoded_2'], encoded_data_df['encoded_3'], c=base_cadastral_scaled[col], cmap='coolwarm')
    plt.title(col)
plt.show()

In [ ]:
print('Each encoded client and its id:')
encoded_data_df['id_cliente'] = base_cadastral['id_cliente']
encoded_data_df

## Preparo para Modelagem
- junção dos dados e feature engineering.
- testagem de modelos de prateleira --> regressão logistica tem melhor resultado.
- modelo de deteção de anomalia

In [ ]:
## Junção dos dados e feature engineering

# statisticaly significant data is lost in the followin merges
base_info_join = base_info_date.merge(encoded_data_df, on='id_cliente', how='inner')
base_join = base_info_join.merge(base_pagamentos_date, on=['id_cliente', 'safra_ref'], how='inner')
base_join_date_ordered = base_join.sort_values(by=['safra_ref'])

def encode_dates_given_ref(df, columns, ref):
    df_copy = df.copy()
    for col in columns:
        min_date = ref
        df_copy[col + '_since_min'] = (df_copy[col] - min_date).dt.days + 1
        df_copy[col + '_year'] = df_copy[col].dt.year
        df_copy[col + '_month'] = df_copy[col].dt.month
        df_copy[col + '_day'] = df_copy[col].dt.day
        df_copy.drop(col, inplace=True, axis=1)
    return df_copy

MIN_DATE = base_join_date_ordered['data_emissao_documento'].min()
base_join_date = encode_dates_given_ref(base_join_date_ordered, ['data_emissao_documento', 'data_vencimento'], MIN_DATE)

# feature engineering
base_join_date['renda_por_funcionario'] = base_join_date['renda_mes_anterior'] / base_join_date['no_funcionarios'].apply(lambda x: x if x > 0 else 1)
base_join_date['valor_a_pagar_por_renda'] = base_join_date['valor_a_pagar'] / base_join_date['renda_mes_anterior']
base_join_date['duracao_emprestimo'] = base_join_date['data_vencimento_since_min'] - base_join_date['data_emissao_documento_since_min']
# add number of loans late by client at the time of the loan / total number of loans taken by the client at the time of the loan
# add number of frauds by client at the time of the loan / total number of loans taken by the client at the time of the loan

X = base_join_date.drop(['id_cliente', 'safra_ref', 'data_pagamento', 'late_payment', 'fraud'], axis=1)
y = base_join_date['fraud']

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_eval = scaler.transform(X_eval)


In [ ]:
## testagem de modelos de prateleira --> regressão logistica tem melhor resultado


from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# Define a list of classifiers to be used
classifiers = [
    (LogisticRegression(class_weight='balanced'), {
        'C': (1e-3, 1e+2, 'log-uniform'),
        'solver': ['lbfgs', 'liblinear']
    }),
    (DecisionTreeClassifier(class_weight='balanced'), {
        'criterion': ['gini', 'entropy'],
        'max_depth': (1, 50),
        'min_samples_split': (2, 10)
    }),
    (RandomForestClassifier(n_estimators=10, class_weight='balanced'), {
        'max_depth': (1, 50),
        'min_samples_split': (2, 10)
    }),
    (LinearSVC(class_weight='balanced', random_state=42), {
        'C': (1e-3, 1e+2, 'log-uniform'),
        'loss': ['hinge', 'squared_hinge']
    })
]

# Define a function to train and evaluate the models
def train_eval_models(clfs, X_train, y_train, X_eval, y_eval, filename):
    accuracy, precision, recall, f1, roc_auc = [], [], [], [], []
    best_models = {}
    for i, (clf, params) in enumerate(clfs):
        clf_name = clf.__class__.__name__

        # Define the BayesSearchCV
        opt = BayesSearchCV(clf, params, n_iter=25, cv=3, n_jobs=-1) # change n_iter to 50 does not improve the results significantly

        # Train the classifier
        opt.fit(X_train, y_train)
        
        # Make predictions on the eval set
        y_pred = opt.predict(X_eval)

        # Save the best model
        best_models[clf_name] = opt.best_estimator_
        
        # Calculate the metrics
        accuracy.append(accuracy_score(y_eval, y_pred))
        precision.append(precision_score(y_eval, y_pred))
        recall.append(recall_score(y_eval, y_pred))
        f1.append(f1_score(y_eval, y_pred))
        roc_auc.append(roc_auc_score(y_eval, y_pred))

        # Print the best parameters and the metrics
        print(f'Best parameters for {clf_name}: {opt.best_params_}')
        print(f'Accuracy: {accuracy[-1]:.2f}')
        print(f'Precision: {precision[-1]:.2f}')
        print(f'Recall: {recall[-1]:.2f}')
        print(f'F1: {f1[-1]:.2f}')
        print(f'ROC AUC: {roc_auc[-1]:.2f}')
        print('-----------------------------------------------------')

    return best_models

# Call the function
best_models = train_eval_models(classifiers, X_train, y_train, X_eval, y_eval, filename='results.txt')
best_models


In [ ]:
## modelo de deteção de anomalia
from sklearn.ensemble import IsolationForest
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

class IsolationForestWrapper(BaseEstimator):
    def __init__(self, n_estimators=10, max_samples='auto', contamination=0.1):
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.contamination = contamination
        self.model = IsolationForest(n_estimators=self.n_estimators, 
                                     max_samples=self.max_samples, 
                                     contamination=self.contamination)
        self.y_train = None

    def fit(self, X, y=None):
        self.y_train = (y == 1).astype(int) if y is not None else None
        self.model.fit(X)
        return self

    def predict(self, X):
        pred = self.model.predict(X)
        return (pred == 1).astype(int)

    def score(self, X, y=None):
        if y is not None:
            y = (y == 1).astype(int)
            y_pred = self.predict(X)
            return f1_score(y, y_pred)
        elif self.y_train is not None:
            y_pred = self.predict(X)
            return f1_score(self.y_train, y_pred)
        else:
            raise ValueError("No y values were given during fitting, cannot calculate score.")

    def set_params(self, **params):
        super().set_params(**params)
        self.model.set_params(**params)
        return self

iso_forest = IsolationForestWrapper(n_estimators=30)

params = {
    'contamination': np.linspace(0.01, 0.2, 20)
}

opt = BayesSearchCV(iso_forest, params, n_iter=25, cv=3, n_jobs=-1)

opt.fit(X_train, y_train)

y_pred = opt.predict(X_eval)

print(confusion_matrix(y_eval, y_pred))

accuracy = accuracy_score(y_eval, y_pred)
precision = precision_score(y_eval, y_pred)
recall = recall_score(y_eval, y_pred)
f1 = f1_score(y_eval, y_pred)
roc_auc = roc_auc_score(y_eval, y_pred)

# Print the best parameters and the metrics
print(f"Best parameters for IsolationForest: {opt.best_params_}")
print(f"Classifier: IsolationForest")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1: {f1:.2f}")
print(f"ROC AUC: {roc_auc:.2f}")
print('-----------------------------------------------------')


## Modelo de Rede Neural
- esse modelo possui o melhor resultado.
- o afinamento dos hyperparametros pode ainda deixa-lo melhor.
- novas features, como a proporção de fraudes por cliente trariam maior banefício.

In [ ]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.metrics import AUC, Precision, Recall
from tensorflow.keras.losses import BinaryFocalCrossentropy


# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X)

histories = []
params_list = []


In [ ]:


def create_layers(input_layer, units, activation='relu', dropout_rate=0.1):
    x_dense = Dense(units, activation=activation)(input_layer)
    x_bn = BatchNormalization()(x_dense)
    x_drop = Dropout(dropout_rate)(x_bn)
    return [x_dense, x_bn, x_drop]

def fraud_detection_model(input_dim, units=15, learning_rate=0.001, weight_decay=0.0001, loss_function='binary_crossentropy'):
    input_layer = Input(shape=(input_dim, ))

    # encoder
    X = create_layers(input_layer, units)
    X_skip = create_layers(X[2], units)
    X = create_layers(X_skip[2] + X[1], units)
    X_skip = create_layers(X[2], units)
    X = create_layers(X_skip[2] + X[1], units)
    X = create_layers(X[2], units)
    X = Dense(1, activation='sigmoid')(X[2])

    model = Model(inputs=input_layer, outputs=X)

    adam = Adam(learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(optimizer=adam, loss=loss_function, metrics=[AUC(), Precision(), Recall()])

    return model

input_dim = X_train.shape[1]

# use bayes optimization to find the best combination of lr, wd, class_weight, and loss function
params = {
    'units': 90,
    'lr': 0.002,
    'wd': 0.0003,
    'lf': 'binary_crossentropy',
    'cw': {0:1, 1:4}
}

model = fraud_detection_model(input_dim=input_dim, units=params['units'], learning_rate=params['lr'], weight_decay=params['wd'], loss_function=params['lf'])
history = model.fit(X_train, y, epochs=40, batch_size=512, validation_split=0.2, class_weight=params['cw'])

y_pred = model.predict(X_eval)


In [ ]:
histories.append(history)
params_list.append(params)

# plot losses
fig = plt.figure(figsize=(20, 20))
for i, history in enumerate(histories):
    ax = fig.add_subplot(5, 5, i+1)
    ax.plot(history.history['loss'])
    ax.plot(history.history['val_loss'])
    title = f'u{ params_list[i]["units"]}, lr {params_list[i]["lr"]:.2e}, wd {params_list[i]["wd"]:.2e}, cw {params_list[i]["cw"][1]}'
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'])
plt.show()

# model.summary()

## Prediction
- usa o modelo anterior para obter a probabilidade de inadimplencia

In [ ]:

## Junção dos dados e feature engineering

# statisticaly significant data is lost in the followin merges
base_info_join = base_info_date.merge(encoded_data_df, on='id_cliente', how='inner')
base_join_test = base_info_join.merge(base_pagamentos_teste_date, on=['id_cliente', 'safra_ref'], how='inner')
base_join_test_date_ordered = base_join_test.sort_values(by=['safra_ref'])

base_join_date_test = encode_dates_given_ref(base_join_test_date_ordered, ['data_emissao_documento', 'data_vencimento'], MIN_DATE)

# feature engineering
base_join_date_test['renda_por_funcionario'] = base_join_date_test['renda_mes_anterior'] / base_join_date_test['no_funcionarios'].apply(lambda x: x if x > 0 else 1)
base_join_date_test['valor_a_pagar_por_renda'] = base_join_date_test['valor_a_pagar'] / base_join_date_test['renda_mes_anterior']
base_join_date_test['duracao_emprestimo'] = base_join_date_test['data_vencimento_since_min'] - base_join_date_test['data_emissao_documento_since_min']

X_test = base_join_date_test.drop(['id_cliente', 'safra_ref'], axis=1)

# scale the data
X_test = scaler.transform(X_test)

y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['INADIMPLENTE']
data_and_pred = pd.concat([base_join_date_test, y_pred], axis=1)

final = data_and_pred[['id_cliente', 'safra_ref', 'INADIMPLENTE']]
final.columns = final.columns.str.upper()
final